<a href="https://colab.research.google.com/github/sumusa/gtfs-visualization-r/blob/main/gtfs_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
R.version.string

[1] "R version 4.0.3 (2020-10-10)"

In [6]:
install.packages("ggthemes")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [8]:
install.packages("leaflet")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘gridExtra’, ‘png’, ‘raster’, ‘sp’, ‘viridis’, ‘leaflet.providers’




In [35]:
library("dplyr")
library("ggplot2")
library("ggthemes")
library("leaflet")

#read in the files
routes <- read.csv("https://openmobilitydata-data.s3-us-west-1.amazonaws.com/public/feeds/metrobus-transit/293/20200827/original/routes.txt")
shapes <- read.csv("https://openmobilitydata-data.s3-us-west-1.amazonaws.com/public/feeds/metrobus-transit/293/20200827/original/shapes.txt")
stops <- read.csv("https://openmobilitydata-data.s3-us-west-1.amazonaws.com/public/feeds/metrobus-transit/293/20200827/original/stops.txt")

# read GTFS stop_times and calendar
calendar <- read.csv("https://openmobilitydata-data.s3-us-west-1.amazonaws.com/public/feeds/metrobus-transit/293/20200827/original/calendar.txt")
trips <- read.csv("https://openmobilitydata-data.s3-us-west-1.amazonaws.com/public/feeds/metrobus-transit/293/20200827/original/trips.txt")
stop_times <- read.csv("https://raw.githubusercontent.com/sumusa/gtfs-visualization-r/main/stop_times.txt")

In [36]:
head(stop_times)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
,<int>,<chr>,<chr>,<int>,<int>,<lgl>,<int>,<int>,<dbl>,<int>
1,185488,7:40:00,7:40:00,1000,1,NA,0,0,NA,1
2,185488,7:41:34,7:41:34,1010,2,NA,0,0,0.7651,0
3,185488,7:41:54,7:41:54,1015,3,NA,0,0,0.9339,0
4,185488,7:42:31,7:42:31,1020,4,NA,0,0,1.2333,0
5,185488,7:43:05,7:43:05,1025,5,NA,0,0,1.5145,0
6,185488,7:43:32,7:43:32,1030,6,NA,0,0,1.7336,0


In [37]:
stops_freq = 
        inner_join(stop_times,stops,by=c("stop_id")) %>%
        inner_join(trips,by=c("trip_id")) %>%
        inner_join(calendar,by=c("service_id")) %>%
        select(stop_id,stop_name,stop_lat,stop_lon) %>%
        group_by(stop_id,stop_name,stop_lat,stop_lon) %>%
        summarize(count=n()) %>%
        filter(count >= 50)

`summarise()` regrouping output by 'stop_id', 'stop_name', 'stop_lat' (override with `.groups` argument)



In [40]:
m <-leaflet() %>%
        # Add CartoDB background map
        addProviderTiles("CartoDB.Positron") %>% 
        addTiles() %>%
        # Add a marker for each stop and route path
        addPolylines (lng= ~shape_pt_lon, lat= ~shape_pt_lat, data = shapes,
                      weight = 2, label = ~shape_pt_sequence, popup = ~shape_pt_sequence, 
                      stroke = TRUE, fillOpacity = 0.5, color = "purple") %>%
        addCircleMarkers(lng= ~ stop_lon, lat= ~stop_lat, data = stops_freq, 
                         popup = ~stop_name, label = ~stop_name, weight = 1, stroke = FALSE, 
                         fillOpacity = 0.5, color = "navy", radius = ~sqrt(count)*0.5)
m

HTML widgets cannot be represented in plain text (need html)